# 1번) 결측치 제거

In [147]:
import numpy as np
import pandas as pd

df=pd.read_csv('Y2019Q4.csv', header=0, engine='python')
df.head()

,지역,망,측정소코드,측정소명,측정일시,SO2,CO,O3,NO2,PM10,PM25,주소
0,서울 중구,도시대기,111121,중구,2019040101,0.002,0.4,0.037,0.022,27.0,16.0,서울특별시 중구 덕수궁길 15
1,서울 중구,도시대기,111121,중구,2019040102,0.003,0.4,0.037,0.021,25.0,15.0,서울특별시 중구 덕수궁길 15
2,서울 중구,도시대기,111121,중구,2019040103,0.003,0.4,0.041,0.016,23.0,13.0,서울특별시 중구 덕수궁길 15
3,서울 중구,도시대기,111121,중구,2019040104,0.002,0.5,0.030,0.025,22.0,12.0,서울특별시 중구 덕수궁길 15
4,서울 중구,도시대기,111121,중구,2019040105,0.002,0.4,0.032,0.023,23.0,12.0,서울특별시 중구 덕수궁길 15


In [148]:
#실전에서 칼럼네임이 한글이면 깨지는 경우가 많습니다. 영어로 변경해봅시다
df=df.rename(index=str, columns={'지역':"SGG","측정소코드":"id","측정소명":"id_name","측정일시":"date","주소":"add"})
df.tail()

,SGG,망,id,id_name,date,SO2,CO,O3,NO2,PM10,PM25,add
314334,인천 옹진군,국가배경농도,831492,백령도,2019043020,0.0017,0.2,0.054,0.0091,16.0,12.0,인천 옹진군 백령면 연화리
314335,인천 옹진군,국가배경농도,831492,백령도,2019043021,0.0013,0.2,0.056,0.0066,19.0,10.0,인천 옹진군 백령면 연화리
314336,인천 옹진군,국가배경농도,831492,백령도,2019043022,0.0012,0.2,0.049,0.0080,21.0,9.0,인천 옹진군 백령면 연화리
314337,인천 옹진군,국가배경농도,831492,백령도,2019043023,0.0010,0.2,0.048,0.0063,21.0,15.0,인천 옹진군 백령면 연화리
314338,인천 옹진군,국가배경농도,831492,백령도,2019043024,0.0010,0.2,0.046,0.0052,22.0,11.0,인천 옹진군 백령면 연화리


In [149]:
#필요한 컬럼들만 뽑아냅시다(참고: PM10은 미세먼지, PM25는 초미세먼지를 의미)
cols=['SGG','id','date','SO2','CO','PM10','PM25']
df2=df[cols]
df2.tail()

,SGG,id,date,SO2,CO,PM10,PM25
314334,인천 옹진군,831492,2019043020,0.0017,0.2,16.0,12.0
314335,인천 옹진군,831492,2019043021,0.0013,0.2,19.0,10.0
314336,인천 옹진군,831492,2019043022,0.0012,0.2,21.0,9.0
314337,인천 옹진군,831492,2019043023,0.0010,0.2,21.0,15.0
314338,인천 옹진군,831492,2019043024,0.0010,0.2,22.0,11.0


In [150]:
#결측치 확인
df2.isna().sum()

SGG         0
id          0
date        0
SO2     12825
CO      13450
PM10    11758
PM25    30374
dtype: int64

In [151]:
df2['SO2']=df2['SO2'].fillna(df2['SO2'].mean())
df2['CO']=df2['CO'].fillna(df2['CO'].mean())
df2.isna().sum()

C:\anaconda\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.
C:\anaconda\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


SGG         0
id          0
date        0
SO2         0
CO          0
PM10    11758
PM25    30374
dtype: int64

In [152]:
df2=df2.dropna(subset=['PM10','PM25'])
df2.isna().sum()

SGG     0
id      0
date    0
SO2     0
CO      0
PM10    0
PM25    0
dtype: int64

# 2번) 새로운 칼럼 생성

In [153]:
#'date' 칼럼의 타입을 datetime으로 설정하고, 이를 인덱스로 설정해봅시다.
##'date'를 연-월-일 형식으로 바꿔봅시다. 마지막 2자리는 제외(마지막 2자리는 hour)
s_date=df2.date.astype(str)
s_date2=s_date.str[:-2]
df2['DATE']=pd.to_datetime(s_date2)
df2.head()

,SGG,id,date,SO2,CO,PM10,PM25,DATE
0,서울 중구,111121,2019040101,0.002,0.4,27.0,16.0,2019-04-01
1,서울 중구,111121,2019040102,0.003,0.4,25.0,15.0,2019-04-01
2,서울 중구,111121,2019040103,0.003,0.4,23.0,13.0,2019-04-01
3,서울 중구,111121,2019040104,0.002,0.5,22.0,12.0,2019-04-01
4,서울 중구,111121,2019040105,0.002,0.4,23.0,12.0,2019-04-01


In [154]:
df2['hour']=s_date.str[-2:]
df2.tail()

,SGG,id,date,SO2,CO,PM10,PM25,DATE,hour
314334,인천 옹진군,831492,2019043020,0.0017,0.2,16.0,12.0,2019-04-30,20
314335,인천 옹진군,831492,2019043021,0.0013,0.2,19.0,10.0,2019-04-30,21
314336,인천 옹진군,831492,2019043022,0.0012,0.2,21.0,9.0,2019-04-30,22
314337,인천 옹진군,831492,2019043023,0.0010,0.2,21.0,15.0,2019-04-30,23
314338,인천 옹진군,831492,2019043024,0.0010,0.2,22.0,11.0,2019-04-30,24


# 3번) 종합문제

In [162]:
df3=df2[df2.PM25>35]

In [163]:
df3.tail()

,SGG,id,date,SO2,CO,PM10,PM25,DATE,hour
313978,인천 옹진군,831492,2019041524,0.0007,0.3,64.0,37.0,2019-04-15,24
314189,인천 옹진군,831492,2019042419,0.0013,0.6,46.0,42.0,2019-04-24,19
314190,인천 옹진군,831492,2019042420,0.0012,0.6,56.0,52.0,2019-04-24,20
314191,인천 옹진군,831492,2019042421,0.0014,0.5,69.0,49.0,2019-04-24,21
314192,인천 옹진군,831492,2019042422,0.0013,0.4,85.0,38.0,2019-04-24,22


In [164]:
df3=df3[['SGG','PM25','hour']]
df3.tail()

,SGG,PM25,hour
313978,인천 옹진군,37.0,24
314189,인천 옹진군,42.0,19
314190,인천 옹진군,52.0,20
314191,인천 옹진군,49.0,21
314192,인천 옹진군,38.0,22


In [170]:
SGG_group=df3.groupby('SGG')['hour']
SGG_group.head()

104       09
105       10
106       11
269       06
270       07
778       11
780       13
800       09
801       10
802       11
1545      10
1546      11
1728      01
1729      02
1730      03
2264      09
2265      10
2266      11
2430      07
2432      09
4377      10
4399      08
4422      07
4423      08
4424      09
5096      09
5097      10
5119      08
5120      09
5121      10
          ..
302851    09
302947    09
302949    11
302950    12
302951    13
303594    08
303595    09
303596    10
303617    07
303618    08
306705    23
306706    24
306707    01
306708    02
306709    03
307194    08
307195    09
307271    13
307272    14
307274    16
308660    10
308661    11
308682    08
308683    09
308684    10
313719    05
313720    06
313721    07
313722    08
313723    09
Name: hour, Length: 1046, dtype: object

In [171]:
df4=df3[['SGG','hour','SGG_group']]
df4.head()

KeyError: "['SGG_group'] not in index"